In [335]:
import pandas as pd

df = pd.read_csv('../data/all_data_for_10000_lines.csv')

## Ajout de la note pondérée

In [336]:
df.head()

# Paramètres pour le calcul du score pondéré
C = df['averagerating'].mean()  # Score moyen de tous les films
m = 1000  # Nombre minimum de votes requis pour être pris en compte

# Calcul du score pondéré
def weighted_rating(x, m=m, C=C):
    v = x['numvotes']
    R = x['averagerating']
    return (v / (v + m) * R) + (m / (v + m) * C)

df['weighted_score'] = df.apply(weighted_rating, axis=1)


In [337]:
df['tconst'][~df['tconst'].isna()].head().values

array(['tt12605172', 'tt12605176', 'tt12605178', 'tt1260518',
       'tt12605180'], dtype=object)

In [338]:
df.describe()

,startyear,endyear,runtimeminutes,averagerating,numvotes,seasonnumber,episodenumber,regionnumber,weighted_score
count,7388.000000,107.000000,2333.000000,1027.000000,1027.000000,6692.000000,6692.000000,10000.00000,1027.000000
mean,2007.504602,2007.130841,40.952422,6.956767,606.494645,3.455021,531.684549,4.72080,6.926801
std,19.236246,16.892046,37.221751,1.385817,6620.788704,7.686266,1507.838716,3.46122,0.326370
min,1906.000000,1951.000000,1.000000,1.100000,5.000000,1.000000,0.000000,1.00000,4.405601
25%,2005.000000,1997.500000,17.000000,6.300000,11.000000,1.000000,7.000000,1.00000,6.943240
50%,2015.000000,2015.000000,30.000000,7.200000,19.000000,1.000000,50.000000,3.00000,6.959651
75%,2020.000000,2020.000000,54.000000,7.900000,64.500000,2.000000,335.000000,8.00000,6.974027
max,2025.000000,2023.000000,435.000000,10.000000,177953.000000,82.000000,13897.000000,50.00000,8.683838


## Identifier les collone numériques et Catégorielles

In [339]:
df.info()

# Les colonnes numériques identifiées sont : 
# startyear, endyear, runtimeminutes, averagerating, numvotes, seasonnumber, episodenumber, weighted_score, isadult.

# Les colonnes catégorielles identifiées sont : 
# tconst, titletype, primarytitle, genres, regionlist, actor, self, producer, actress, director.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          10000 non-null  object 
 1   titletype       10000 non-null  object 
 2   primarytitle    10000 non-null  object 
 3   isadult         10000 non-null  bool   
 4   startyear       7388 non-null   float64
 5   endyear         107 non-null    float64
 6   runtimeminutes  2333 non-null   float64
 7   genres          9776 non-null   object 
 8   averagerating   1027 non-null   float64
 9   numvotes        1027 non-null   float64
 10  seasonnumber    6692 non-null   float64
 11  episodenumber   6692 non-null   float64
 12  regionnumber    10000 non-null  int64  
 13  regionlist      10000 non-null  object 
 14  actor           9195 non-null   object 
 15  self            9195 non-null   object 
 16  producer        9195 non-null   object 
 17  actress         9195 non-null   

## fonctions préprocessing

In [340]:
import nltk
from nltk import PorterStemmer
nltk.download('wordnet')
nltk.download('wordnet')

#fonction de stemming

def stemming(liste):
  stemming = []
  for element in liste:
    elementStemme = PorterStemmer().stem(element)
    stemming.append(elementStemme)
  return stemming


import re
import string


#mettre en minuscule et supprimer les caractères spéciaux et les espaces en début et fin de texte 

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s\[\]]', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\p2972\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\p2972\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Préprocessing collone TitleType

In [341]:
df['titletype'].value_counts()

# La colonne 'titletype' ne présente pas de valeurs incohérentes.
# Elle n'a pas besoin d'être tokenisée, stemmée, ni traitée pour les stop words, 
# mais elle nécessite d'être encodée.


titletype
tvEpisode       7887
short            821
movie            402
video            289
tvSeries         271
tvMovie          159
tvSpecial         65
videoGame         53
tvMiniSeries      44
tvShort            9
Name: count, dtype: int64

In [342]:
df['titletype'].value_counts()
#Utiliser ordinal encoder pour encoder la colonne titletype
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
OrdinalEncoder_titletype = encoder.fit_transform(df['titletype'].values.reshape(-1, 1)) #reshape(-1, 1) pour convertir une liste en tableau



## Préprocessing collone Primarytitle

In [343]:
df['primarytitle'].head().values

# On remarque que les titres sont variés et ne se ressemblent pas. Une tokenisation est donc nécessaire.

# Les stop words ne doivent pas être supprimés, car cela pourrait altérer le sens des titres.
# Certains titres sont définis par leurs stop words, par exemple : "The Walking Dead" ou "The Good Place".

# Un stemming est recommandé. Par exemple, si un utilisateur apprécie un film contenant le mot "run", 
# le stemming ou la lemmatisation permettra de faire correspondre ce mot avec d'autres variantes telles que "running" ou "runs". 
# Cela améliore la qualité des recommandations basées sur la similitude des titres.

# Pour un système simple basé sur la présence ou l'absence de mots, le TF-IDF peut être approprié.
# Il représente chaque titre en fonction des mots qu'il contient, avec un poids plus élevé pour les mots rares et significatifs.
# Cela peut aider à identifier les mots associés aux meilleures notes, analyser leur influence, ou améliorer la gestion des titres similaires.


array(['Episode #1.2', 'Shaheb Bibi Golam', 'Girl in the Mirror',
       'Loose Ends', 'Episode #1.3'], dtype=object)

In [344]:
#Collone primarytitle

#Stemming
df['primarytitle_processed'] = df['primarytitle'].apply(stemming)

#Retirer caractères spéciaux, espaces et retourner en minuscule
df['primarytitle_processed'] = df['primarytitle'].apply(clean_text)

#Tokenization
df['primarytitle_processed'] = df['primarytitle_processed'].apply(lambda x : x.split())

df[['primarytitle_processed','primarytitle']].head()






from sklearn.feature_extraction.text import TfidfVectorizer

# 1. Rejointure des tokens en une chaîne de texte
df['primarytitle_processed'] = df['primarytitle_processed'].apply(lambda x: ' '.join(x))

# 1. Initialisation de TfidfVectorizer et vectorisation
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['primarytitle_processed'])

# 2. Conversion de la matrice TF-IDF en DataFrame avec préfixe
tfidf_df_primary_title = pd.DataFrame(tfidf_matrix.toarray(), columns=['tfidf_' + word for word in vectorizer.get_feature_names_out()])




## Preprocessing collone genres

In [345]:
df['genres']

0             Action,Mystery
1                      Drama
2                Music,Short
3                      Drama
4             Action,Mystery
                ...         
9995    Crime,Drama,Thriller
9996               Adventure
9997                   Drama
9998                   Adult
9999                  Sci-Fi
Name: genres, Length: 10000, dtype: object

In [346]:
df['genres'].value_counts()

# On observe 354 combinaisons de genres différentes.
# Il est nécessaire de les tokeniser, et de les encoder.


genres
Drama                    2203
Talk-Show                 722
Comedy                    598
Documentary               537
Reality-TV                425
                         ... 
Crime,Horror,Mystery        1
Action,Comedy,Fantasy       1
Reality-TV,Sport            1
Action,Drama,Mystery        1
Horror,Mystery              1
Name: count, Length: 354, dtype: int64

In [347]:
df['genres'].apply(lambda x : type(x)).value_counts() #On compte les données de type str

df[df['genres'].apply(lambda x: isinstance(x, float))] #On vérifie les valeurs nulles

,tconst,titletype,primarytitle,isadult,startyear,endyear,runtimeminutes,genres,averagerating,numvotes,...,episodenumber,regionnumber,regionlist,actor,self,producer,actress,director,weighted_score,primarytitle_processed
36,tt1260526,tvEpisode,Hank Williams Jr.,False,1979.0,NaN,NaN,NaN,NaN,NaN,...,NaN,1,['\\N'],[],[],[],[],['Steven North'],NaN,hank williams jr
41,tt1260527,tvEpisode,Ricky Nelson,False,1979.0,NaN,NaN,NaN,NaN,NaN,...,NaN,1,['\\N'],[],['Ricky Nelson'],[],[],['Steven North'],NaN,ricky nelson
127,tt1260544,tvEpisode,Episode #1.16,False,2008.0,NaN,90.0,NaN,NaN,NaN,...,16.0,8,"['\\N', 'PT', 'IN', 'FR', 'IT', 'DE', 'JP', 'ES']",[],"['Juha Helppi', 'Robin Keston', 'Dave Mattey',...",[],[],[],NaN,episode 116
236,tt12605676,tvSeries,CBC News: At Issue,False,2007.0,NaN,30.0,NaN,3.6,7.0,...,NaN,2,"['\\N', 'CA']",[],"['Althia Raj', 'Althia Raj', 'Rosemary Barton'...",[],[],[],6.933433,cbc news at issue
291,tt12605782,tvEpisode,Episode #1.11,False,2020.0,NaN,21.0,NaN,NaN,NaN,...,11.0,8,"['\\N', 'PT', 'IN', 'FR', 'IT', 'DE', 'JP', 'ES']",NaN,NaN,NaN,NaN,NaN,NaN,episode 111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9743,tt12623656,tvMovie,A tocar!,False,2020.0,NaN,NaN,NaN,NaN,NaN,...,NaN,2,"['\\N', 'ES']",[],"['Camille Decourtye', 'Frederic Amat', 'Blaï M...",[],[],"['Camille Decourtye', 'Blaï Mateu']",NaN,a tocar
9797,tt1262377,tvEpisode,Episode #1.7,False,2008.0,NaN,NaN,NaN,NaN,NaN,...,7.0,8,"['\\N', 'PT', 'IN', 'FR', 'IT', 'DE', 'JP', 'ES']","['David Ramírez', 'Toni Mora']","['Jordi Borràs', 'Carles Rexach', 'Anna Llache...",[],[],"['Òscar Lorca', 'Mai Balaguer']",NaN,episode 17
9801,tt1262378,tvEpisode,Episode #5.40,False,2008.0,NaN,NaN,NaN,NaN,NaN,...,40.0,8,"['\\N', 'PT', 'IN', 'FR', 'IT', 'DE', 'JP', 'ES']",[],"['Susanna Griso', 'Ferran Monegal']",[],[],['Ferran Monegal'],NaN,episode 540
9864,tt12623922,tvEpisode,Episode dated 1 August 2005,False,2005.0,NaN,NaN,NaN,NaN,NaN,...,NaN,8,"['\\N', 'PT', 'IN', 'FR', 'IT', 'DE', 'JP', 'ES']",[],"['José Miguel Viñuela', 'Jessica Cirio', 'Thia...",[],[],[],NaN,episode dated 1 august 2005


In [349]:
# 1. Créer une copie de la colonne 'genres' pour le traitement
df['genres_processed'] = df['genres']

# 2. Supprimer les valeurs de type float en convertissant en chaîne de caractères
df['genres_processed'] = df['genres_processed'].astype(str)

# 3. Analyser les genres : compter les occurrences de chaque genre individuel
df['genres_processed'].apply(lambda x: x.split(',')).explode().value_counts()

# 4. Appliquer MultiLabelBinarizer sur la colonne 'genres_processed'
from sklearn.preprocessing import MultiLabelBinarizer

# Initialisation du binariseur
mlb = MultiLabelBinarizer()

# Conversion de la colonne en listes de genres
df['genres_processed'] = df['genres_processed'].apply(lambda x: x.split(','))

# Appliquer le MultiLabelBinarizer pour créer une matrice binaire
multilabel = mlb.fit_transform(df['genres_processed'])


multilabel

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(10000, 28))